# Stress-Strain Using Strongly-Typed GPSR

## Goal: Show how simple stress-strain relationships can be learned using Bingo's strongly-typed functionality

### Pre-Requisites

It is recommended to check out the [symbolic regression tutorial](tutorial_4.html) before continuing.

### Problem Description

We can describe the relationship between stress and strain as $\mathbf{\sigma} =
C \mathbf{\epsilon}$, where $\mathbf{\sigma}$ is a stress tensor,
$\mathbf{\epsilon}$ is a strain tensor, and $C$ is a tensor of constants
according to the anisotropy of the material. We can use Bingo's strongly typed
functionality to learn this relationship, as well as more complicated tensor
relationships.

### Creating Training Data

#### Getting C In Voigt Form

We can define anisotropies based on the number of unique constants in Voigt
form and where those constants show up.

In the case of cubic symmetry, there are 3 unique constants, with the first
and third constant showing up on the main diagonal, and the second constant
showing up on the second and third diagonals:

In [1]:
import numpy as np

ANISOTROPY_STRUCTURES = {"cubic": [3, np.array([[1, 2, 2, 0, 0, 0],
                                                [2, 1, 2, 0, 0, 0],
                                                [2, 2, 1, 0, 0, 0],
                                                [0, 0, 0, 3, 0, 0],
                                                [0, 0, 0, 0, 3, 0],
                                                [0, 0, 0, 0, 0, 3]])]}

We can then define a function that gets a random $C$ matrix based on a
particular anistropy defined in `ANISOTROPY_STRUCTURES`:

In [2]:
def get_random_c_matrix(anisotropy):
    if anisotropy.lower() not in ANISOTROPY_STRUCTURES.keys():
        raise RuntimeError(f"Unsupported anistropy: {anisotropy.lower()}")

    # get info from mapping
    n_constants, structure = ANISOTROPY_STRUCTURES[anisotropy]

    random_constants = np.random.rand(n_constants)

    # get indices of constants (e.g., 1, 2, ...)
    constant_idxs = []
    for i in range(1, 1+n_constants):
        constant_idxs.append(structure == i)

    # use indices to replace 1 with first random constant, and so forth
    c_matrix = np.zeros((6, 6))
    for i, constant_idx in enumerate(constant_idxs):
        c_matrix[constant_idx] = random_constants[i]

    return c_matrix

#### Generating Training Data

We can generate random $\mathbf{\epsilon}$ tensors in Voigt form and get
their respective $\mathbf{\sigma}$ tensors to train on.

In [3]:
np.random.seed(0)

N_POINTS = 100

epsilon = np.random.rand(N_POINTS, 6, 6)

C = get_random_c_matrix("cubic")
print(f"Trying to find: sigma = \n{C}(epsilon)")

sigma = C @ epsilon

Trying to find: sigma = 
[[0.25746648 0.95859445 0.95859445 0.         0.         0.        ]
 [0.95859445 0.25746648 0.95859445 0.         0.         0.        ]
 [0.95859445 0.95859445 0.25746648 0.         0.         0.        ]
 [0.         0.         0.         0.8180574  0.         0.        ]
 [0.         0.         0.         0.         0.8180574  0.        ]
 [0.         0.         0.         0.         0.         0.8180574 ]](epsilon)


Formally creating the training data is as simple as making epsilon the
first variable, and sigma the output:

In [4]:
from bingo.symbolic_regression.explicit_regression_md import ExplicitTrainingDataMD

training_data = ExplicitTrainingDataMD([epsilon], sigma)

C:\Users\David\Desktop\GPSR Research\bingoNASAFork\bingo\symbolic_regression\__init__.py:31: UserWarning: Could not load C++ modules cannot import name 'AGraph' from 'bingocpp' (unknown location)
  warnings.warn(f"Could not load C++ modules {import_err}")


### Setting Up the Evolutionary Process

Setting up the evolutionary process for strongly-typed GPSR is very similar to setting up Bingo's
normal evolutionary process, as described in [the symbolic regression tutorial](tutorial_4.html).
However, there are some notable differences:

- Objects specific to multiple-dimensions have MD at the end
(e.g., `ExplicitRegressionMD` instead of `ExplicitRegression`)

- Operators relate to linear algebra/tensors instead of scalars

#### Input and Output Shapes

Many strongly-typed objects need type information, so let's store the
shape information of our input and output variables.

In [5]:
inputs, output = training_data.x, training_data.y
input_dims = [np.shape(_x[0]) for _x in inputs]
output_dim = output[0].shape

#### Component Generator

The strongly-typed component generator is basically the same as Bingo's
typical component generator, expect the operators relate to matrices and/or
tensors. Furthermore, we can specify the shapes of possible constants:

In [6]:
from bingo.symbolic_regression.agraphMD.component_generator_md import ComponentGeneratorMD

component_generator = ComponentGeneratorMD(input_dims, possible_dims=[(6, 6)])
component_generator.add_operator("+")
component_generator.add_operator("*")

In this case, we are only using matrix addition and multiplication. We are
also only allowing for constants to be 6x6.


#### Crossover and Mutation

In [7]:
from bingo.symbolic_regression.agraphMD.crossover_md import AGraphCrossoverMD
from bingo.symbolic_regression.agraphMD.mutation_md import AGraphMutationMD

crossover = AGraphCrossoverMD()
mutation = AGraphMutationMD(component_generator, command_probability=0.333,node_probability=0.333,
                            parameter_probability=0.333, prune_probability=0.0, fork_probability=0.0)

#### AGraph Generator

In [8]:
from bingo.symbolic_regression.agraphMD.generator_md import AGraphGeneratorMD

STACK_SIZE = 6

agraph_generator = AGraphGeneratorMD(STACK_SIZE, component_generator, input_dims, output_dim, use_simplification=False)

#### Evaluation

Fitness functions are specific to multiple-dimensions (i.e., have MD after
object names). However, evaluation is normal.

In [9]:
from bingo.symbolic_regression.explicit_regression_md import ExplicitRegressionMD
from bingo.local_optimizers.continuous_local_opt_md import ContinuousLocalOptimizationMD
from bingo.evaluation.evaluation import Evaluation

fitness = ExplicitRegressionMD(training_data=training_data)
local_opt_fitness = ContinuousLocalOptimizationMD(fitness, algorithm='lm', param_init_bounds=[0, 1])
evaluator = Evaluation(local_opt_fitness)

#### Evolutionary Algorithm and Optimizer

These objects are not specific to multiple-dimensions, so normal objects
can be used.

In [10]:
from bingo.evolutionary_algorithms.age_fitness import AgeFitnessEA
from bingo.evolutionary_optimizers.island import Island

POP_SIZE = 100

ea = AgeFitnessEA(evaluator, agraph_generator, crossover,
                  mutation, 0.4, 0.4, POP_SIZE)

island = Island(ea, agraph_generator, POP_SIZE)

generating population
finished generating population


### Evolution

In [11]:
import random
random.seed(0)

MAX_GENS = 500
FITNESS_THRESHOLD = 1e-6


opt_result = island.evolve_until_convergence(max_generations=MAX_GENS,
                                             fitness_threshold=FITNESS_THRESHOLD)

Generation: 0, Best fitness: 0.14313117181713214


### Results

In [12]:
np.set_printoptions(suppress=True)  # print w/o scientific notation
print(opt_result)
best_indiv = island.get_best_individual()
print(best_indiv.get_formatted_string("console"))
print(best_indiv.fitness)

OptimizeResult(success=True, status=0, message='Absolute convergence occurred with best fitness < 1e-06', ngen=1, fitness=5.751358118942471e-10, time=29.747764, ea_diagnostics=EaDiagnosticsSummary(beneficial_crossover_rate=0.0, detrimental_crossover_rate=0.5, beneficial_mutation_rate=0.35294117647058826, detrimental_mutation_rate=0.4411764705882353, beneficial_crossover_mutation_rate=0.3333333333333333, detrimental_crossover_mutation_rate=0.16666666666666666))
([[-0.29626349 -1.27113864  3.04207341  0.00000001 -0.00000001 -0.        ]
 [ 1.20968297  1.80166261 -1.5366743  -0.          0.          0.        ]
 [ 0.5612518   0.94414731 -0.03072784 -0.         -0.          0.        ]
 [-0.         -0.          0.          1.35248166 -1.40339667  0.45332252]
 [-0.         -0.          0.          0.64903869 -1.12862712  0.65672562]
 [-0.         -0.          0.         -0.22123433  0.69300928  0.68061072]])(([[-0.29626349 -1.27113864  3.04207341  0.00000001 -0.00000001 -0.        ]
 [ 1.2